In [40]:
from langchain.llms import OpenAI
from langchain import HuggingFaceHub
import os
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain, SequentialChain


In [14]:
load_dotenv()

True

In [20]:
OPENAI_API_KEY_VAR = os.getenv('OPENAI_API_KEY')
TEMPERATURE = 0.6

In [12]:
llm = OpenAI(openai_api_key = OPENAI_API_KEY_VAR, temperature=TEMPERATURE)

NameError: name 'TEMPERATURE' is not defined

In [13]:
test_text = "Capital of poland is ?"
print(llm.predict(test_text))



Warsaw


In [21]:
TEMPERATURE

0.6

In [24]:
llm_huggingface = HuggingFaceHub(huggingfacehub_api_token = os.getenv('HUGGINGFACE_API_KEY'), repo_id="google/flan-t5-large", model_kwargs={"temperature": TEMPERATURE, "max_length": 64})

In [25]:
test_text = "Capital of poland is ?"
print(llm_huggingface.predict(test_text))

Warsaw


In [33]:
prompt_template = PromptTemplate(
    input_variables=["country"], template="tell me the capital of {country}"
)

In [34]:
prompt_template.format(country="poland")

'tell me the capital of poland'

In [35]:
chain = LLMChain(llm=llm, prompt = prompt_template)
print(chain.run("Poland"))



The capital of Poland is Warsaw.


In [38]:
capital_template = PromptTemplate(
    input_variables=["country"], template="tell me the capital of {country}"
)
capital_chain = LLMChain(llm=llm, prompt = capital_template)

famous_template = PromptTemplate(input_variables=["capital"],
                template="Suggest me some amazing places to visit in {capital}")
famous_chain = LLMChain(llm=llm, prompt = famous_template)

In [39]:
chain = SimpleSequentialChain(chains=[capital_chain, famous_chain])
chain.run("Poland")

" There are many amazing places to visit in Warsaw, including:\n\n1. Old Town (Stare Miasto) - This is the historic heart of Warsaw, with beautiful cobblestone streets, colorful buildings, and charming cafes.\n\n2. Palace of Culture and Science - This iconic building, built in the Soviet style, offers stunning views of the city from its observation deck.\n\n3. Warsaw Uprising Museum - This museum tells the story of the Warsaw Uprising during World War II and is a must-visit for history buffs.\n\n4. Lazienki Park - This is the largest park in Warsaw, with beautiful gardens, a palace, and even a free Chopin concert on Sundays in the summer.\n\n5. Royal Castle - The former residence of Polish monarchs, this castle now houses a museum with impressive art collections and historical artifacts.\n\n6. Wilanow Palace - Another beautiful palace to visit, with stunning gardens and a museum showcasing the history of Poland's aristocracy.\n\n7. POLIN Museum of the History of Polish Jews - This mode

In [42]:
capital_template = PromptTemplate(
    input_variables=["country"], template="tell me the capital of {country}"
)
capital_chain = LLMChain(llm=llm, prompt = capital_template, output_key="capital")

famous_template = PromptTemplate(input_variables=["capital"],
                template="Suggest me some amazing places to visit in {capital}")
famous_chain = LLMChain(llm=llm, prompt = famous_template, output_key="places")

chain = SequentialChain(
    chains=[capital_chain, famous_chain],
    input_variables=['country'],
    output_variables=['capital', 'places']
)

In [43]:
chain({"country": "india"})

/media/kornellewy/jan_dysk_3/nlp_projects/nlp_venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


{'country': 'india',
 'capital': '\n\nThe capital of India is New Delhi.',
 'places': " Some amazing places to visit in New Delhi are:\n\n1. Red Fort: A historic fort built during the Mughal era, known for its stunning architecture and light and sound show.\n\n2. Qutub Minar: A UNESCO World Heritage Site, this 73-meter tall minaret is a must-visit for its intricate carvings and rich history.\n\n3. India Gate: A war memorial and iconic landmark of New Delhi, it is a popular spot for picnics and evening strolls.\n\n4. Humayun's Tomb: Another UNESCO World Heritage Site, this magnificent tomb is a fusion of Mughal and Persian architecture.\n\n5. Lotus Temple: A Bahá'í House of Worship, this stunning lotus-shaped structure is a peaceful place for meditation and reflection.\n\n6. Akshardham Temple: A relatively modern temple known for its grand architecture, intricate carvings, and multimedia shows.\n\n7. Jama Masjid: The largest mosque in India, it is an impressive example of Mughal archite

In [48]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage

In [51]:
chatllm = ChatOpenAI(openai_api_key = OPENAI_API_KEY_VAR, temperature=TEMPERATURE, model='gpt-3.5-turbo')

/media/kornellewy/jan_dysk_3/nlp_projects/nlp_venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [57]:
chatllm([
    SystemMessage(content = "You are a comedian ai assistant."),
    HumanMessage(content = "Please write 3 panchlines.")
])

AIMessage(content='1. "I tried to be a stand-up comedian, but I kept falling flat on my face...literally!"\n2. "My love life is like a bad joke: it\'s always punchline me in the gut!"\n3. "I used to be a baker, but I couldn\'t rise to the occasion. I guess I kneaded more dough!"')

In [58]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import BaseOutputParser 

In [59]:
class CommaSeparatedOutput(BaseOutputParser):
    def parse(self, text: str):
        return text.strip().strip(",")

In [63]:
template = "Your are a helpful assistant. When the use given any input , you should generate 5 words synonyms in a comma seperated list"
human_template = "{text}"
chatprompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template),
])

In [64]:
chain = chatprompt|chatllm|CommaSeparatedOutput()

In [65]:
chain.invoke({"text": "ai"})

'artificial intelligence, machine learning, automation, robotics, cognitive computing'